In [1]:
from datetime import datetime, timedelta
from sqlalchemy import create_engine, MetaData, Table
from scrape_sofa import get_sofascore_api_data, init_driver, get_all_players_stats_fbref, get_all_teams_stats_fbref
from funciones import get_stat
import pandas as pd

today = datetime.now()
yesterday = today - timedelta(days=1)
yesterday = yesterday.strftime('%Y-%m-%d')
engine = create_engine('mysql+pymysql://root@localhost/footviz')
df_partidos_ayer = pd.read_sql(f"SELECT * FROM partido WHERE fecha = '{yesterday}'", engine)


In [4]:
driver = init_driver()
data = get_sofascore_api_data(driver, path='api/v1/player/814594/unique-tournament/34/season/77356/heatmap/overall')

In [13]:
data['points'][0]['x']

67

In [15]:
puntos_x = []
puntos_y = []
for i in range(len(data['points'])):
    puntos_x.append(data['points'][i]['x'])
    puntos_y.append(data['points'][i]['y'])


print(min(puntos_x))
print(min(puntos_y))

0
0
